<a href="https://colab.research.google.com/github/rahulrajpr/AdvancedFeatureEngineering/blob/DataImputation/DataImputation_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [2]:
path = '/content/drive/MyDrive/Udemy Courses - 2.0/Feature Engineering/Dataset/houseprice/houseprice.csv'
df = pd.read_csv(path)
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
df.shape

(1460, 81)

In [4]:
!pip install feature_engine
clear_output()

In [5]:
cols_to_use = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']

In [6]:
data = df[cols_to_use]

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BsmtQual     1423 non-null   object 
 1   FireplaceQu  770 non-null    object 
 2   LotFrontage  1201 non-null   float64
 3   MasVnrArea   1452 non-null   float64
 4   GarageYrBlt  1379 non-null   float64
 5   SalePrice    1460 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 68.6+ KB


In [8]:
data.isnull().mean()

BsmtQual       0.025342
FireplaceQu    0.472603
LotFrontage    0.177397
MasVnrArea     0.005479
GarageYrBlt    0.055479
SalePrice      0.000000
dtype: float64

In [9]:
X = data.drop('SalePrice', axis =1)
y = data['SalePrice']

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7, random_state = 100)

In [12]:
X_train.shape,X_test.shape

((1021, 5), (439, 5))

In [13]:
num_cols = X.select_dtypes(include = np.number).columns.tolist()
obj_cols = X.select_dtypes(exclude = np.number).columns.tolist()

Feature Engine Median Imputation

In [14]:
from feature_engine.imputation import MeanMedianImputer

In [15]:
mmi = MeanMedianImputer(imputation_method = 'median',variables = num_cols)

In [16]:
mmi.fit(X_train)

MeanMedianImputer(variables=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])

In [17]:
mmi.variables_

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [18]:
mmi.imputer_dict_

{'LotFrontage': 68.0, 'MasVnrArea': 0.0, 'GarageYrBlt': 1981.0}

In [19]:
X_train = mmi.transform(X_train)
X_train

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
318,Gd,TA,90.0,256.0,1993.0
239,TA,Gd,52.0,0.0,1949.0
986,TA,NaN,59.0,0.0,1950.0
1416,TA,NaN,60.0,0.0,1971.0
390,TA,NaN,50.0,0.0,1978.0
...,...,...,...,...,...
802,Gd,Gd,63.0,0.0,2005.0
53,Ex,Gd,68.0,0.0,1981.0
350,Ex,Gd,68.0,362.0,2007.0
79,TA,NaN,60.0,0.0,1966.0


In [20]:
X_test = mmi.transform(X_test)

Feature Engine Mean Imputation

In [21]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size = 0.7, random_state = 100)

In [22]:
mmi_medain = MeanMedianImputer(imputation_method = 'mean', variables = num_cols)

In [23]:
mmi_medain.fit(X_train)

MeanMedianImputer(imputation_method='mean',
                  variables=['LotFrontage', 'MasVnrArea', 'GarageYrBlt'])

In [24]:
X_train = mmi_medain.transform(X_train)
X_test = mmi_medain.transform(X_test)

Feature Engine in combination with Sklearn Pipeline

In [25]:
from sklearn.pipeline import Pipeline

In [26]:
cols_for_mean = ['LotFrontage']
cols_for_median = ['MasVnrArea','GarageYrBlt']

In [27]:
pipe = Pipeline(steps = [('mean_impute',MeanMedianImputer(imputation_method = 'mean', variables = cols_for_mean)),
                         ('median_impute',MeanMedianImputer(imputation_method = 'median', variables = cols_for_median))])

In [28]:
pipe.fit(X_train)

Pipeline(steps=[('mean_impute',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['LotFrontage'])),
                ('median_impute',
                 MeanMedianImputer(variables=['MasVnrArea', 'GarageYrBlt']))])

In [29]:
X_train = pipe.transform(X_train)

In [30]:
X_test = pipe.transform(X_test)

In [31]:
X_train.isnull().mean()

BsmtQual       0.028404
FireplaceQu    0.484819
LotFrontage    0.000000
MasVnrArea     0.000000
GarageYrBlt    0.000000
dtype: float64

End of trail imputation in Feature Engine

In [32]:
from feature_engine.imputation import EndTailImputer

In [33]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']

In [34]:
data = df[sel_cols]

In [35]:
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']

In [36]:
X_train,x_test,y_train,y_test = train_test_split(X,y, train_size = 0.7, random_state = 100)

Gaussian Distribution

In [37]:
ET_imputer_gaussian =  EndTailImputer(imputation_method = 'gaussian',tail = 'right')

In [38]:
ET_imputer_gaussian.fit(X_train)

EndTailImputer()

In [39]:
ET_imputer_gaussian.imputer_dict_

{'LotFrontage': 142.58106404871148,
 'MasVnrArea': 667.6842622489197,
 'GarageYrBlt': 2052.7303260689037}

In [40]:
X_train = ET_imputer_gaussian.transform(X_train)
X_test = ET_imputer_gaussian.transform(X_test)

IQR Distribution

In [41]:
X_train,x_test,y_train,y_test = train_test_split(X,y, train_size = 0.7, random_state = 100)

In [42]:
ET_imputer_iqr = EndTailImputer(imputation_method = 'iqr', tail = 'left')

In [43]:
ET_imputer_iqr.fit(X_train)

EndTailImputer(imputation_method='iqr', tail='left')

In [44]:
ET_imputer_iqr.imputer_dict_

{'LotFrontage': -4.0, 'MasVnrArea': -474.0, 'GarageYrBlt': 1842.0}

In [45]:
X_train = ET_imputer_iqr.transform(X_train)
X_test = ET_imputer_iqr.transform(X_test)

End of Tail Imputation with Pipeline

In [46]:
X_train,x_test,y_train,y_test = train_test_split(X,y, train_size = 0.7, random_state = 100)

In [47]:
var_set1 = ['GarageYrBlt', 'MasVnrArea']
var_set2 = ['LotFrontage']

In [48]:
pipe2 = Pipeline([('imputer_gaussian',EndTailImputer(imputation_method = 'gaussian',tail = 'right', variables = var_set1)),
                  ('imputer_iqr',EndTailImputer(imputation_method = 'iqr',tail = 'right', variables = var_set2))])

In [49]:
pipe2.fit(X_train)

Pipeline(steps=[('imputer_gaussian',
                 EndTailImputer(variables=['GarageYrBlt', 'MasVnrArea'])),
                ('imputer_iqr',
                 EndTailImputer(imputation_method='iqr',
                                variables=['LotFrontage']))])

In [50]:
X_train = pipe2.transform(X_train)

In [51]:
X_test = pipe2.transform(X_test)

### Categorical Data Imputation

Most Frequent Imputation - Mode Imputation

In [52]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']
data = df[sel_cols]
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
X_train,X_test, y_train,y_test = train_test_split(X,y,train_size = 0.70, random_state = 100)

In [53]:
from feature_engine.imputation import CategoricalImputer

In [54]:
catImp = CategoricalImputer(imputation_method = 'frequent')

In [55]:
catImp.fit(X_train)

CategoricalImputer(imputation_method='frequent')

In [56]:
catImp.imputer_dict_

{'BsmtQual': 'TA', 'FireplaceQu': 'Gd'}

In [57]:
X_train = catImp.transform(X_train)
X_test = catImp.transform(X_test)

In [58]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021 entries, 318 to 792
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BsmtQual     1021 non-null   object 
 1   FireplaceQu  1021 non-null   object 
 2   LotFrontage  844 non-null    float64
 3   MasVnrArea   1017 non-null   float64
 4   GarageYrBlt  964 non-null    float64
dtypes: float64(3), object(2)
memory usage: 47.9+ KB


'Misiing' - String Imputation

In [59]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']
data = df[sel_cols]
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
X_train,X_test, y_train,y_test = train_test_split(X,y,train_size = 0.70, random_state = 100)

In [60]:
from feature_engine.imputation import CategoricalImputer

In [61]:
cat_missing = CategoricalImputer(imputation_method= 'missing')

In [62]:
X_train['BsmtQual'].value_counts()

TA    452
Gd    428
Ex     87
Fa     25
Name: BsmtQual, dtype: int64

In [63]:
X_train['BsmtQual'].isnull().sum()

29

In [64]:
cat_missing.fit(X_train)

CategoricalImputer()

In [65]:
X_train = cat_missing.transform(X_train)
X_test = cat_missing.transform(X_test)

In [66]:
X_train['BsmtQual'].value_counts()

TA         452
Gd         428
Ex          87
Missing     29
Fa          25
Name: BsmtQual, dtype: int64

In [67]:
X_train['BsmtQual'].isnull().sum()

0

### Random Sample Imputation

In [68]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']
data = df[sel_cols]
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
X_train,X_test, y_train,y_test = train_test_split(X,y,train_size = 0.70, random_state = 100)

In [69]:
from feature_engine.imputation import RandomSampleImputer

In [70]:
rand_sample_imput = RandomSampleImputer(seeding_method = 'add')

In [71]:
rand_sample_imput.fit(X_train)

RandomSampleImputer()

In [72]:
rand_sample_imput.X_

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
318,Gd,TA,90.0,256.0,1993.0
239,TA,Gd,52.0,0.0,1949.0
986,TA,NaN,59.0,0.0,1950.0
1416,TA,NaN,60.0,0.0,1971.0
390,TA,NaN,50.0,0.0,1978.0
...,...,...,...,...,...
802,Gd,Gd,63.0,0.0,2005.0
53,Ex,Gd,68.0,0.0,1981.0
350,Ex,Gd,68.0,362.0,2007.0
79,TA,NaN,60.0,0.0,1966.0


In [73]:
X_train = rand_sample_imput.transform(X_train)

In [74]:
X_train.isnull().sum()

BsmtQual       0
FireplaceQu    0
LotFrontage    0
MasVnrArea     0
GarageYrBlt    0
dtype: int64

### Adding Missing data indicator for the features

In [75]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']
data = df[sel_cols]
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
X_train,X_test, y_train,y_test = train_test_split(X,y,train_size = 0.70, random_state = 100)

In [77]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1021 entries, 318 to 792
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   BsmtQual     992 non-null    object 
 1   FireplaceQu  526 non-null    object 
 2   LotFrontage  844 non-null    float64
 3   MasVnrArea   1017 non-null   float64
 4   GarageYrBlt  964 non-null    float64
dtypes: float64(3), object(2)
memory usage: 47.9+ KB


In [80]:
X_train.isnull().sum()

BsmtQual        29
FireplaceQu    495
LotFrontage    177
MasVnrArea       4
GarageYrBlt     57
dtype: int64

In [79]:
from feature_engine.imputation import AddMissingIndicator

In [82]:
missin_ind = AddMissingIndicator(missing_only =True)

In [83]:
missin_ind.fit(X_train)

AddMissingIndicator()

In [84]:
missin_ind.transform(X_train)

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt,BsmtQual_na,FireplaceQu_na,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
318,Gd,TA,90.0,256.0,1993.0,0,0,0,0,0
239,TA,Gd,52.0,0.0,1949.0,0,0,0,0,0
986,TA,NaN,59.0,0.0,1950.0,0,1,0,0,0
1416,TA,NaN,60.0,0.0,1971.0,0,1,0,0,0
390,TA,NaN,50.0,0.0,1978.0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
802,Gd,Gd,63.0,0.0,2005.0,0,0,0,0,0
53,Ex,Gd,68.0,0.0,1981.0,0,0,0,0,0
350,Ex,Gd,68.0,362.0,2007.0,0,0,0,0,0
79,TA,NaN,60.0,0.0,1966.0,0,1,0,0,0


Even when you add a missing indicator, we need to still impute the original columns inorder to make it usable for learning

In [88]:
sel_cols = ['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt','SalePrice']
data = df[sel_cols]
X = data.drop('SalePrice', axis = 1)
y = data['SalePrice']
X_train,X_test, y_train,y_test = train_test_split(X,y,train_size = 0.70, random_state = 100)

In [89]:
num_cols = X_train.select_dtypes(include = np.number).columns.tolist()
obj_cols = X_train.select_dtypes(exclude = np.number).columns.tolist()

In [90]:
pipe = Pipeline([('missing_indicator', AddMissingIndicator(missing_only = True)),
                 ('mean_imput',MeanMedianImputer(imputation_method = 'mean', variables = num_cols)),
                 ('frequent_imput',CategoricalImputer(imputation_method = 'frequent', variables = obj_cols))])

In [91]:
pipe.fit(X_train)

Pipeline(steps=[('missing_indicator', AddMissingIndicator()),
                ('mean_imput',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['LotFrontage', 'MasVnrArea',
                                              'GarageYrBlt'])),
                ('frequent_imput',
                 CategoricalImputer(imputation_method='frequent',
                                    variables=['BsmtQual', 'FireplaceQu']))])

In [93]:
pipe.named_steps['missing_indicator'].variables_

['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [94]:
pipe.named_steps['mean_imput'].variables_

['LotFrontage', 'MasVnrArea', 'GarageYrBlt']

In [95]:
pipe.named_steps['frequent_imput'].variables_

['BsmtQual', 'FireplaceQu']

In [97]:
X_train = pipe.transform(X_train)
X_train

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt,BsmtQual_na,FireplaceQu_na,LotFrontage_na,MasVnrArea_na,GarageYrBlt_na
318,Gd,TA,90.0,256.0,1993.0,0,0,0,0,0
239,TA,Gd,52.0,0.0,1949.0,0,0,0,0,0
986,TA,Gd,59.0,0.0,1950.0,0,1,0,0,0
1416,TA,Gd,60.0,0.0,1971.0,0,1,0,0,0
390,TA,Gd,50.0,0.0,1978.0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...
802,Gd,Gd,63.0,0.0,2005.0,0,0,0,0,0
53,Ex,Gd,68.0,0.0,1981.0,0,0,0,0,0
350,Ex,Gd,68.0,362.0,2007.0,0,0,0,0,0
79,TA,Gd,60.0,0.0,1966.0,0,1,0,0,0


In [99]:
X_test = pipe.transform(X_test)

In [98]:
X_train.isnull().sum()

BsmtQual          0
FireplaceQu       0
LotFrontage       0
MasVnrArea        0
GarageYrBlt       0
BsmtQual_na       0
FireplaceQu_na    0
LotFrontage_na    0
MasVnrArea_na     0
GarageYrBlt_na    0
dtype: int64